In [ ]:
%env MKL_NUM_THREADS=1

import os
from functools import partial

import numpy as np
from scipy import sparse
from sklearn.feature_extraction import text

import implicit

from ya_cup_2022.scoring import mrr_score

In [ ]:
TRAIN_FN = 'train'
TEST_FN = '_test_xxl'
ANS_FN = '_ans_xxl'

N = 100

WD = os.path.join('D:', 'education', 'yaintern', 'yandex_cup_2022', 'data')
WD

In [ ]:
def vectorize(frain_file, test_file):
    vectorizer = text.CountVectorizer(lowercase=False, token_pattern=r'\b\d+\b')
    
    with open(frain_file, 'rt') as f:
        users = [line.strip() for line in f.readlines()]
    
    test_users_offset = len(users)
    
    with open(test_file, 'rt') as f:
        users.extend(line.strip() for line in f.readlines())
        
    X = vectorizer.fit_transform(users)
    return X, vectorizer, test_users_offset

In [ ]:
X, vectorizer, test_offset = vectorize(
    os.path.join(WD, TRAIN_FN),
    os.path.join(WD, TEST_FN),
)

X

In [ ]:
ind_2_track = dict()
for track, ind in vectorizer.vocabulary_.items():
    ind_2_track[ind] = track

In [ ]:
with open(os.path.join(WD, ANS_FN), 'rt') as f:
    y_true = [line.strip() for line in f.readlines()]

In [ ]:
model = implicit.bpr.BayesianPersonalizedRanking(
    factors=100,
#     alpha=100.0,
#     regularization=10.0,
)

model.fit(X)
recs = model.recommend(
    list(range(test_offset, X.shape[0])),
    X[test_offset:, :],
    filter_already_liked_items=True,
    N=N,
)

mrr_score(y_true, [[ind_2_track[i] for i in rec] for rec in recs[0]])